In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px


df_temperature = pd.read_csv("../data/temperature-quotidienne-departementale.csv", sep=';')
df_temperature.rename(columns = {"TMin (°C)": "TMin"}, inplace=True)
df_temperature.rename(columns = {"TMax (°C)": "TMax"}, inplace=True)
df_temperature.rename(columns = {"TMoy (°C)": "TMoy"}, inplace=True)
#We only consider dates before 2023, so dates between 2014 and 2022
df_temperature = df_temperature[df_temperature.Date<"2023"]
df_temperature.head()

,Date,Code INSEE département,Département,TMin,TMax,TMoy
0,2019-10-22,21,Côte-d'Or,10.90,16.00,13.45
1,2019-10-22,73,Savoie,11.70,18.65,15.18
2,2019-10-22,29,Finistère,7.77,15.48,11.63
3,2019-10-22,48,Lozère,9.70,16.50,13.10
4,2019-10-22,49,Maine-et-Loire,4.40,15.10,9.75


In [49]:
# Create a dictionary to map agglomerations to departments
agglomeration_department_map = {
    'Agglomération de Brest': ['29'],
    'Agglomération de Châteauroux': ['36'],
    'Agglomération de Draguignan': ['83'],
    'Agglomération de Fréjus': ['83'],
    'Agglomération de Nancy': ['54'],
    'Agglomération de Nîmes': ['30'],
    'Agglomération de Toulon': ['83'],
    'Agglomération de Toulouse': ['31'],
    'Auray Quiberon Terre Atlantique': ['56'],
    'Golfe du Morbihan - Vannes Agglomération': ['56'],
    'Lorient Agglomération': ['56'],
    'Agglomération d\'Aix-Marseille': ['13'],
    'Agglomération d\'Alençon': ['61'],
    'Agglomération d\'Arles': ['13'],
    'Agglomération de Besançon': ['25'],
    'Agglomération de Lille': ['59'],
    'Agglomération de Lyon': ['69'],
    'Agglomération de Montpellier': ['34'],
    'Agglomération de Nantes': ['44'],
    'Agglomération de Nice-Menton': ['06'],
    'Agglomération de Rennes': ['35'],
    'Agglomération de Saint Denis (La Réunion)': ['974'],
    'Agglomération de Sète': ['34'],
    'Agglomération parisienne': ['75', '77', '78', '91', '92', '93', '94', '95'],
    'Agglomération parisienne (hors Paris)': ['77', '78', '91', '92', '93', '94', '95'],
    'Paris intra-muros': ['75'],
    'Sud Loire - proche Haute-Loire': ['42'],
    'Unité urbaine de l\'agglomération de Bayonne': ['64']
}

In [50]:
department_map = dict(e[1:]
                      for e in df_temperature[["Code INSEE département", "Département"]].drop_duplicates().itertuples())

In [51]:
agglomeration_department_map = {key:list(map(department_map.get, value))
                                for key, value in agglomeration_department_map.items()}

In [52]:
month_dic1 = {'01':'Jan', '02':'Feb', '03':'Mar', '04':'Apr', '05':'May', '06':'Jun', '07':'Jul', '08':'Aug', '09':'Sep',
             '10':'Oct', '11':'Nov', '12':'Dec'}
month_dic2 = {'Apr': 4, 'Aug': 8, 'Dec': 12, 'Feb': 2, 'Jan': 1, 'Jul': 7, 'Jun': 6, 'Mar': 3, 'May': 5, 'Nov': 11,
              'Oct': 10, 'Sep': 9}
def date_conversion(date):
    date_split = date.split("-")
    return f"{month_dic1[date_split[1]]} {date_split[0]}"

In [53]:
departements_selectionnes = ['29', '36', '83', '54', '30', '31', '56', '13', '61', '25', '59', '69', '34', '44', '06',
                             '35', '974', '75', '42', '43', '64', '77', "95"]
df_temperature = df_temperature[df_temperature['Code INSEE département'].isin(departements_selectionnes)]
df_temperature

,Date,Code INSEE département,Département,TMin,TMax,TMoy
2,2019-10-22,29,Finistère,7.77,15.48,11.63
5,2019-10-23,83,Var,18.56,22.28,20.42
6,2019-10-23,36,Indre,11.40,17.50,14.45
11,2019-10-23,42,Loire,11.75,14.90,13.32
20,2019-10-23,35,Ille-et-Vilaine,9.40,15.35,12.38
...,...,...,...,...,...,...
175274,2022-12-09,34,Hérault,5.93,14.73,10.33
175281,2022-12-10,56,Morbihan,-0.20,6.60,3.20
175283,2022-12-10,64,Pyrénées-Atlantiques,0.83,7.33,4.08
175290,2022-12-11,54,Meurthe-et-Moselle,-3.90,-1.00,-2.45


In [54]:
df_temperature_departments = df_temperature[["Département", "TMin", "TMax", "TMoy"]].groupby("Département")\
.mean().sort_values(by="TMoy").reset_index()
df_temperature_departments.head()

,Département,TMin,TMax,TMoy
0,Haute-Loire,4.560296,15.694962,10.127629
1,Meurthe-et-Moselle,7.026205,16.594688,11.810487
2,Orne,7.514294,16.758434,12.136364
3,Nord,8.916670,15.639814,12.278193
4,Doubs,7.669387,17.241457,12.455422


In [55]:
fig = px.bar(df_temperature_departments, x="TMoy", y="Département", title="Average Temperature by Department",
             labels={"TMoy":"Average Temperature (°C)", "Département":"Department"}, orientation='h')
fig.update_traces(hovertemplate="%{label} : <br> %{value:.2f} °C")
fig.show()

In [56]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_temperature_departments["TMoy"],
                     y=df_temperature_departments["Département"],
                     orientation='h',
                     error_x = {"arrayminus":df_temperature_departments["TMoy"]-df_temperature_departments["TMin"],
                                "array":df_temperature_departments["TMax"]-df_temperature_departments["TMoy"]}
                    )
             )
fig.update_traces(hovertemplate="%{label} : <br> %{value:.2f} °C<extra></extra>")
fig.update_layout(xaxis_title = "Average Temperature (°C)",
                  yaxis_title="Department",
                  title={"text":"Average Temperature by Department", "x":0.5, "xanchor":"center"})
fig.show()

In [57]:
df_temperature["Mois"] = df_temperature["Date"].str.rsplit("-", 1).str[0].apply(date_conversion)
df_temperature["Année"] = df_temperature["Mois"].str.split(" ").str[1]

In [58]:
avg_temperatures = df_temperature.groupby(["Mois", "Année", "Département"]).mean().reset_index()
avg_temperatures_year = {annee:avg_temperatures[avg_temperatures["Année"]==annee]
                     for annee in ("2018", "2019", "2020", "2021", "2022")}

In [59]:
departments = df_temperature_departments["Département"]
data_temperature_evolution = {
    (year, department): temperature[temperature["Département"]==department]\
    .sort_values(by="Mois", key=lambda serie: [month_dic2[e.split()[0]] for e in serie])
    for year, temperature in avg_temperatures_year.items()
    for department in departments
}

In [60]:
charts_temperature_evolution = {
    (year, department): px.bar(data,
                            x="Mois",
                            y="TMoy",
                            labels={"Mois":"Month", "TMoy":"Average Temperature (°C)"},
                            title=f"Evolution of the average temperature in {department} accross months of {year}")
    for (year, department), data in data_temperature_evolution.items()
}

In [61]:
list(charts_temperature_evolution.values())[10]

In [62]:
#This structure is done to avoid a FutureWarning ("The default dtype for empty Series will be 'object' instead of
#'float64' in a future version. Specify a dtype explicitly to silence this warning.")
data_temperature_evolution_agglomeration = {
    key:value.groupby("Mois").mean().reset_index().sort_values(by="Mois",
                                                               key=lambda serie: [month_dic2[e.split()[0]] for e in serie])
    for key, value in {
        (year, agglomeration): temperature[temperature["Département"].isin(departments)]
        for year, temperature in avg_temperatures_year.items()
        for agglomeration, departments in agglomeration_department_map.items()
    }.items()
    if len(value)
}

In [63]:
charts_temperature_evolution_agglomeration = {
    (year, agglomeration): px.bar(data,
                            x="Mois",
                            y="TMoy",
                            labels={"Mois":"Month", "TMoy":"Average Temperature (°C)"},
                            title=f"Evolution of the average temperature in {agglomeration} accross months of {year}")
    for (year, agglomeration), data in data_temperature_evolution_agglomeration.items()
}

In [64]:
list(charts_temperature_evolution_agglomeration.values())[50]

In [65]:
list(data_temperature_evolution_agglomeration.values())[50]

,Mois,TMin,TMax,TMoy
4,Jan 2019,1.602419,6.052258,3.827581
3,Feb 2019,1.890357,12.420179,7.155357
7,Mar 2019,4.788226,13.740806,9.263710
0,Apr 2019,5.432000,16.603667,11.017833
8,May 2019,7.791290,18.141935,12.966774
6,Jun 2019,13.388833,24.933167,19.161333
5,Jul 2019,14.327581,28.001613,21.164677
1,Aug 2019,14.181935,26.481129,20.330806
11,Sep 2019,11.068833,22.305667,16.687000
10,Oct 2019,10.112742,16.553548,13.333226


In [ ]:
agglomeration_rent_map = {}